## Deliverable 2. Create a Customer Travel Destinations Map.

In [11]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [12]:
# 1. Import the WeatherPy_database.csv file. 

city_data_df= pd.read_csv("Weather_Database/WeatherPy_Database.csv")

city_data_df.head()

,WeatherPy_Database.csv,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Port Alfred,ZA,-33.5906,26.8910,60.19,91,95,4.00,overcast clouds
1,1,Yulara,AU,-25.2406,130.9889,50.02,49,0,9.37,clear sky
2,2,Anchorage,US,61.2181,-149.9003,55.38,90,100,15.01,light rain
3,3,Ribeira Grande,PT,38.5167,-28.7000,75.60,88,40,6.91,scattered clouds
4,4,Bluff,NZ,-46.6000,168.3333,42.80,73,89,14.45,overcast clouds


In [13]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.

min_temp = float(input("What is the minimum temperature you would like for your trip? "))

max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 65
What is the maximum temperature you would like for your trip? 80


In [14]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.

preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,WeatherPy_Database.csv,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
3,3,Ribeira Grande,PT,38.5167,-28.7000,75.60,88,40,6.91,scattered clouds
11,11,Saint-Philippe,RE,-21.3585,55.7679,67.59,79,41,21.43,light rain
15,15,Puerto Ayora,EC,-0.7393,-90.3518,78.76,90,72,5.99,broken clouds
20,20,San Quintin,MX,30.4833,-115.9500,75.69,63,100,9.78,overcast clouds
27,27,Vaini,TO,-21.2000,-175.2000,73.56,94,75,10.36,broken clouds
28,28,Gaurela,IN,22.7500,81.9000,72.03,96,100,5.30,overcast clouds
29,29,Praya,CV,14.9215,-23.5087,79.34,83,20,9.22,few clouds
35,35,Jiangyou,CN,31.7667,104.7167,69.73,93,46,1.01,scattered clouds
36,36,Marsa Matruh,EG,31.3525,27.2453,79.00,73,40,11.50,scattered clouds
41,41,Bandarbeyla,SO,9.4942,50.8122,75.18,84,50,35.05,scattered clouds


In [15]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

WeatherPy_Database.csv    0
City                      0
Country                   0
Lat                       0
Lng                       0
Max Temp                  0
Humidity                  0
Cloudiness                0
Wind Speed                0
Description               0
dtype: int64

In [16]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Get the maximum temperature.

clean_df = preferred_cities_df.dropna()

In [17]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# Get the maximum temperature.

clean_df.count()

WeatherPy_Database.csv    222
City                      222
Country                   222
Lat                       222
Lng                       222
Max Temp                  222
Humidity                  222
Cloudiness                222
Wind Speed                222
Description               222
dtype: int64

In [18]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.

hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"

hotel_df["Hotel Name"] = ""


hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Ribeira Grande,PT,75.60,scattered clouds,38.5167,-28.7000,
11,Saint-Philippe,RE,67.59,light rain,-21.3585,55.7679,
15,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,
20,San Quintin,MX,75.69,overcast clouds,30.4833,-115.9500,
27,Vaini,TO,73.56,broken clouds,-21.2000,-175.2000,
28,Gaurela,IN,72.03,overcast clouds,22.7500,81.9000,
29,Praya,CV,79.34,few clouds,14.9215,-23.5087,
35,Jiangyou,CN,69.73,scattered clouds,31.7667,104.7167,
36,Marsa Matruh,EG,79.00,scattered clouds,31.3525,27.2453,
41,Bandarbeyla,SO,75.18,scattered clouds,9.4942,50.8122,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.


params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [20]:
# Iterate through the DataFrame.

for index, row in hotel_df.iterrows():
    
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    try:  
    # Make request and get the JSON data from the search.
        hotels = requests.get(base_url, params=params).json()
    
    
    # Grab the first hotel from the results and store the name.
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
    except (IndexError):
        print("Hotel not found... skipping.")   

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City           222
Country        222
Max Temp       222
Description    222
Lat            222
Lng            222
Hotel Name     222
dtype: int64

In [28]:
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
3,Ribeira Grande,PT,75.60,scattered clouds,38.5167,-28.7000,Quinta da Meia Eira
11,Saint-Philippe,RE,67.59,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
15,Puerto Ayora,EC,78.76,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
20,San Quintin,MX,75.69,overcast clouds,30.4833,-115.9500,Don Eddie's Sport Fishing Center
27,Vaini,TO,73.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort


In [29]:
# 8a. Create the output File (CSV)

output_data_file = "Weather_Database/WeatherPy_vacation.csv"


# 8b. Export the City_Data into a csv

clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [30]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [31]:
# 11a. Add a marker layer for each city to the map. 
# Add a heatmap of temperature for the vacation spots and marker for each city.


In [26]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))